# Setup Packages

In [ ]:
import os
from pathlib import Path
import sys

# --- REPO ROOT ON sys.path (so `from src.*` works locally) ---
_REPO_ROOT = str(Path(os.getcwd()).resolve().parents[1])
if _REPO_ROOT not in sys.path:
    sys.path.insert(0, _REPO_ROOT)


# --- ENVIRONMENT SWITCH ---
# Set to True if running on local machine with Google Drive Desktop mounted
# Set to False if running in Google Colab cloud
RUNNING_LOCALLY = True

if RUNNING_LOCALLY:
    # Standard macOS path for Google Drive Desktop
    BASE_PATH = Path('/Volumes/GoogleDrive/MyDrive/AI Public Trust')
else:
    # Google Colab cloud path
    from google.colab import drive
    drive.mount('/content/drive')
    BASE_PATH = Path('/content/drive/MyDrive/AI Public Trust')

# Pre-compute critical paths used across notebooks
twits_folder = BASE_PATH / 'Raw Data/Twits/'
test_folder = BASE_PATH / 'Raw Data/'
datasets_folder = BASE_PATH / 'Data Sets'
cleanedds_folder = BASE_PATH / 'Data Sets/Cleaned Data'
networks_folder = BASE_PATH / 'Data Sets/Networks/'
literature_folder = BASE_PATH / 'Literature/'
topic_models_folder = BASE_PATH / 'Models/Topic Modeling/'


In [ ]:
import tweepy
#from tweepy import Stream
from datetime import timedelta
import json
import time
import datetime
import os
from datetime import timedelta
import tqdm
import pickle
import numpy as np
import tqdm
import pandas as pd
import random
from pathlib import Path

# from google.colab import drive
# drive.mount('/content/drive')

# %%
# ──────────────────────────────────────────────────────────────────────────────
# 1. Drive Mount & Paths
# ──────────────────────────────────────────────────────────────────────────────

# Base project folder (Ignacio standard)
# BASE = Path('/content/drive/My Drive/Colab Projects/AI Public Trust')
# BASE = Path('/content/drive/My Drive/AI Public Trust')


# twits_folder = BASE / 'Raw Data/Twits/'
# test_folder = BASE / 'Raw Data/'
# print("Current Directory:", twits_folder)
# datasets_folder = BASE / 'Data Sets/'
# cleanedds_folder = BASE / 'Data Sets' / 'Cleaned Data'

Mounted at /content/drive
Current Directory: /content/drive/My Drive/AI Public Trust/Raw Data/Twits


# Sanity Check

In [ ]:
# LOOKING AT THE HEAD OF THE FILES TO SEE THEIR SHAPE AND STATS
file_list = os.listdir(twits_folder)
print(len(file_list))
file_list[0]

5016


'tweets_2023-02-06T16:30:00.json'

In [ ]:
file_list[:5]

['tweets_2023-02-06T16:30:00.json',
 'tweets_2023-02-06T17:00:00.json',
 'tweets_2023-02-06T17:30:00.json',
 'tweets_2023-02-06T18:00:00.json',
 'tweets_2023-02-06T18:30:00.json']

In [ ]:
file_list[-5:]

['tweets_2022-11-15T05:30:00.json',
 'tweets_2022-11-15T06:00:00.json',
 'tweets_2022-11-15T06:30:00.json',
 'tweets_2022-11-15T07:00:00.json',
 'tweets_2022-11-15T07:30:00.json']

In [ ]:
f = open(test_folder/"testing.json",'r',encoding='utf-8')
test = json.load(f)
f.close()
print('Remember we set the requests limit to '+str(2000)+'.')
print('While the total requests found were of '+str(len(test))+'.')
print('Also we set the max per request at 500, and the data has more or less '+str(len(test[0]['data']))+' twits per request'+'.')

print('--------------- exploring the data ------------')
page=test[0]
data = page['data']
print(type(data))
print(len(data))
print(type(data[0]))
print(data[0].keys())
print(data[0]['text'])
print(data[0]['created_at'])
print(data[0]['referenced_tweets'])

print('--------------- exploring the extensions ------------')
includes = page['includes']
print(type(includes))
print(includes.keys())
print(type(includes['tweets']))
print(len(includes['tweets']))
print(includes['tweets'][0].keys())
print(includes['tweets'][0]['text'])
print(type(includes['users']))
print(len(includes['users']))
print(includes['users'][0].keys())
print(includes['users'][0]['public_metrics'])
print('--------------- exploring the meta ------------')
meta = page['meta']
print(meta)

Remember we set the requests limit to 2000.
While the total requests found were of 3.
Also we set the max per request at 500, and the data has more or less 472 twits per request.
--------------- exploring the data ------------
<class 'list'>
472
<class 'dict'>
dict_keys(['id', 'referenced_tweets', 'text', 'conversation_id', 'author_id', 'edit_history_tweet_ids', 'public_metrics', 'entities', 'lang', 'possibly_sensitive', 'created_at'])
RT @WonderBotz: GENIE automatically captures all user activity and utilizes one-of-a-kind AI to create process flows and actionable insight…
2022-11-15T00:09:59.000Z
[{'type': 'retweeted', 'id': '1592178048002326529'}]
--------------- exploring the extensions ------------
<class 'dict'>
dict_keys(['users', 'tweets'])
<class 'list'>
238
dict_keys(['id', 'entities', 'text', 'conversation_id', 'author_id', 'edit_history_tweet_ids', 'public_metrics', 'lang', 'possibly_sensitive', 'created_at'])
GENIE automatically captures all user activity and utilizes one-

In [ ]:
file = 'tweets_2023-02-07T17:00:00.json'
f = open(twits_folder/file,'r',encoding='utf-8')
twitfile = json.load(f)
f.close()

print('Remember we set the requests limit to '+str(2000)+'.')
print('While the total requests found were of '+str(len(twitfile))+'.')
print('Also we set the max per request at 500, and the data has more or less '+str(len(test[0]['data']))+' twits per request'+'.')
print('\n')
print('--------------- exploring the data ------------')
page=twitfile[0]
data = page['data']
print(type(data))
print(len(data))
print(type(data[0]))
print(data[0].keys())
print(data[0]['text'])
print(data[0]['created_at'])
print(data[0]['referenced_tweets'])
print(data[0]['public_metrics'])
print(data[0]['entities'])

print('\n')
print('--------------- exploring the extensions ------------')
includes = page['includes']
print(type(includes))
print(includes.keys())
print(type(includes['tweets']))
print(len(includes['tweets']))
print(includes['tweets'][0].keys())
print(includes['tweets'][0]['text'])

print(type(includes['users']))
print(len(includes['users']))
print(includes['users'][0].keys())
print(includes['users'][0]['public_metrics'])
print(includes['users'][0]['name'])
print('\n')
print('--------------- exploring the meta ------------')
meta = page['meta']
print(meta)

Remember we set the requests limit to 2000.
While the total requests found were of 20.
Also we set the max per request at 500, and the data has more or less 472 twits per request.


--------------- exploring the data ------------
<class 'list'>
470
<class 'dict'>
dict_keys(['entities', 'referenced_tweets', 'text', 'lang', 'edit_history_tweet_ids', 'public_metrics', 'conversation_id', 'possibly_sensitive', 'author_id', 'id', 'created_at'])
RT @lporiginalg: "nothing forever" a 24-hour twitch live stream that is AI generated Seinfeld was banned after calling transgenders mentall…
2023-02-07T17:29:59.000Z
[{'type': 'retweeted', 'id': '1623004469683425281'}]
{'retweet_count': 124, 'reply_count': 0, 'like_count': 0, 'quote_count': 0, 'impression_count': 0}
{'annotations': [{'start': 45, 'end': 50, 'probability': 0.8822, 'type': 'Other', 'normalized_text': 'twitch'}, {'start': 85, 'end': 92, 'probability': 0.5293, 'type': 'Person', 'normalized_text': 'Seinfeld'}], 'mentions': [{'start': 3, 'e

# Tweet Processing Functions

In [ ]:
def process_tweet(original_tweet):
    # Now we proceed to generate the dictionary and save data
    # we create a dictionary 'd' and put the twit data there, which we will json dump and print at the end
    twit_dict=dict()
    twit_dict['id'] = original_tweet['id']
    twit_dict['text']=original_tweet['text']
    twit_dict['created_at']=original_tweet['created_at']
    twit_dict['public_metrics']=original_tweet['public_metrics']
    twit_dict['author_id']=original_tweet['author_id']
    twit_dict['type']='original'
    try:
      twit_dict['type']=original_tweet['referenced_tweets'][0]['type']
      twit_dict['referenced_tweets']=original_tweet['referenced_tweets'][0]['id']
    except:
      pass
    try:
      twit_dict['conversation_id']=original_tweet['conversation_id']
    except:
      pass
    return twit_dict

# Test Data Procesing

In [ ]:
%%time
AItrust_twits_dict_test = open(cleanedds_folder/'AItrust_twits_dict_test.json','w',encoding='utf-8')
AItrust_author_dict_test = open(cleanedds_folder/'AItrust_author_dict_test.json','w',encoding='utf-8')
count_processed = 0
count_total = 0
count_authors = 0
count_processed_authors = 0
for file in tqdm.tqdm(["testing.json"]):
    f = open(test_folder/file,'r',encoding='utf-8')
    twitfile = json.load(f)
    f.close()
    for twlist in twitfile:

        # Generate dictionary of tweets in extensions in that file
        file_twit_extension_dict=dict()
        try:
          for extension_tweet in twlist['includes']['tweets']:
              try:
                  processed_tweet = process_tweet(extension_tweet)
                  file_twit_extension_dict[processed_tweet['id']]=processed_tweet
              except Exception as e:
                  print(e)
                  pass
        except Exception as e:
          print(e)
          print('\n')
          print('No tweet extensions.')
          print('This happens in file: ' + str(file))
          print('\n')
          pass

        # Now look at the original tweets and add the extension if useful
        file_twit_dict=dict()
        for original_tweet in twlist['data']:
            count_total+=1
            try:
                processed_tweet = process_tweet(original_tweet)
                file_twit_dict[processed_tweet['id']]=processed_tweet
                try:
                    referenced_tw_id = processed_tweet['referenced_tweets']
                    if referenced_tw_id in file_twit_extension_dict.keys():
                        referenced_tw = file_twit_extension_dict[referenced_tw_id]
                        processed_tweet['referenced_tweets_dictionary']=referenced_tw
                    else:
                        processed_tweet['referenced_tweets_dictionary']='N/A'
                except:
                    pass
                dictionary = json.dumps(processed_tweet,ensure_ascii=False)#.encode('utf8')
                print(dictionary,file=AItrust_twits_dict_test)
                count_processed+=1
            except Exception as e:
                print(e)

        # Now see how much you can get from the extensions
            try:
              for extension_tweet in twlist['includes']['tweets']:
                  count_total+=1
                  try:
                      processed_tweet = process_tweet(extension_tweet)
                      try:
                          referenced_tw_id = processed_tweet['referenced_tweets']
                          if referenced_tw_id in file_twit_extension_dict.keys():
                              referenced_tw = file_twit_extension_dict[referenced_tw_id]
                              processed_tweet['referenced_tweets_dictionary']=referenced_tw
                          if referenced_tw_id in file_twit_dict.keys():
                              referenced_tw = file_twit_dict[referenced_tw_id]
                              processed_tweet['referenced_tweets_dictionary']=referenced_tw
                          else:
                              processed_tweet['referenced_tweets_dictionary']='N/A'
                      except:
                          pass
                      dictionary = json.dumps(processed_tweet,ensure_ascii=False)#.encode('utf8')
                      print(dictionary,file=AItrust_twits_dict_test)
                      count_processed+=1
                  except Exception as e:
                      print(e)
                      pass
            except Exception as e:
              print(e)
              pass

        # Now the author
        try:
          for author in twlist['includes']['users']:
            # # dict_keys(['description', 'public_metrics', 'created_at', 'id', 'entities', 'name', 'username', 'verified'])
              count_authors+=1
              try:
                  author_dictionary = json.dumps(author,ensure_ascii=False)#.encode('utf8')
                  print(author_dictionary,file=AItrust_author_dict_test)
                  count_processed_authors+=1
              except Exception as e:
                  print(e)
        except Exception as e:
          print(e)
          print('\n')
          print('No Authors?!!')
          print('This happens in file: ' + str(file))
          print('\n')
          pass


AItrust_twits_dict_test.close()
AItrust_author_dict_test.close()
print('\n')
print('Tweets counts:')
print(count_processed)
print(count_total)
print('\n')
print('Author counts:')
print(count_processed_authors)
print(count_authors)

100%|██████████| 1/1 [00:04<00:00,  4.12s/it]



Tweets counts:
233094
233094


Author counts:
1272
1272
CPU times: user 3.38 s, sys: 117 ms, total: 3.5 s
Wall time: 9.64 s


In [ ]:
# Check how it looks, LINE BY LINE
# CHECKING FOR TYPES OF TWEET THAT ARE NOT ORIGINAL
AItrust_twits_dict_test = open(cleanedds_folder/'AItrust_twits_dict_test.json','r',encoding='utf-8')

i = 0
for line in AItrust_twits_dict_test:
    twit = json.loads(line)
    #print(type(twit))
    #print(twit['text'])
    #print(twit['id'])
    #print(twit['type'])
    try:
      i+=1
      print(twit['referenced_tweets'])
      print(twit['type'])
      print(twit.keys())
      print('------')
    except:
      pass
    #print('------')
    if i>5:
        break
AItrust_twits_dict_test.close()

1592178048002326529
retweeted
dict_keys(['id', 'text', 'created_at', 'public_metrics', 'author_id', 'type', 'referenced_tweets', 'conversation_id', 'referenced_tweets_dictionary'])
------
1592187343351463936
replied_to
dict_keys(['id', 'text', 'created_at', 'public_metrics', 'author_id', 'type', 'referenced_tweets', 'conversation_id', 'referenced_tweets_dictionary'])
------
1591860196737175553
replied_to
dict_keys(['id', 'text', 'created_at', 'public_metrics', 'author_id', 'type', 'referenced_tweets', 'conversation_id', 'referenced_tweets_dictionary'])
------


In [ ]:
# Check how it looks, LINE BY LINE
# dict_keys(['description', 'public_metrics', 'created_at', 'id', 'entities', 'name', 'username', 'verified'])
AItrust_author_dict_test = open(cleanedds_folder/'AItrust_author_dict_test.json','r',encoding='utf-8')

i = 0
for line in AItrust_author_dict_test:
    i+=1
    author = json.loads(line)
    print(author.keys())
    print(author['verified'])
    if i>10:
        break
AItrust_author_dict_test.close()

dict_keys(['description', 'public_metrics', 'created_at', 'id', 'entities', 'name', 'username', 'verified'])
False
dict_keys(['description', 'public_metrics', 'created_at', 'id', 'entities', 'name', 'username', 'verified'])
False
dict_keys(['description', 'public_metrics', 'created_at', 'id', 'entities', 'name', 'username', 'verified', 'location'])
False
dict_keys(['description', 'public_metrics', 'created_at', 'id', 'name', 'username', 'verified'])
False
dict_keys(['description', 'public_metrics', 'created_at', 'id', 'entities', 'name', 'username', 'verified'])
False
dict_keys(['description', 'public_metrics', 'created_at', 'id', 'entities', 'name', 'username', 'verified', 'location'])
True
dict_keys(['description', 'public_metrics', 'created_at', 'id', 'entities', 'name', 'username', 'verified', 'location'])
False
dict_keys(['description', 'public_metrics', 'created_at', 'id', 'name', 'username', 'verified'])
False
dict_keys(['description', 'public_metrics', 'created_at', 'id', 'enti

# Process All Data

In [ ]:
#%%time
AItrust_twits_dict = open(cleanedds_folder/'AItrust_twits_dict.json','w',encoding='utf-8')
AItrust_author_dict = open(cleanedds_folder/'AItrust_author_dict.json','w',encoding='utf-8')
count_processed = 0
count_total = 0
count_authors = 0
count_processed_authors = 0

for file in tqdm.tqdm(file_list):
    f = open(twits_folder/file,'r',encoding='utf-8')
    twitfile = json.load(f)
    f.close()
    for twlist in twitfile:

        # Generate dictionary of tweets in extensions in that file
        try:
            file_twit_extension_dict=dict()
            for extension_tweet in twlist['includes']['tweets']:
                try:
                    processed_tweet = process_tweet(extension_tweet)
                    file_twit_extension_dict[processed_tweet['id']]=processed_tweet
                except Exception as e:
                    print(str(e))
                    pass
        except:
            pass


# Now look at the original tweets and add the extension if useful
        try:
            file_twit_dict=dict()
            for original_tweet in twlist['data']:
                count_total+=1
                try:
                    processed_tweet = process_tweet(original_tweet)
                    file_twit_dict[processed_tweet['id']]=processed_tweet
                    try:
                        referenced_tw_id = processed_tweet['referenced_tweets']
                        if referenced_tw_id in file_twit_extension_dict.keys():
                            referenced_tw = file_twit_extension_dict[referenced_tw_id]
                            processed_tweet['referenced_tweets_dictionary']=referenced_tw
                        else:
                            processed_tweet['referenced_tweets_dictionary']='N/A'
                    except:
                        pass
                    dictionary = json.dumps(processed_tweet,ensure_ascii=False)#.encode('utf8')
                    print(dictionary,file=AItrust_twits_dict)
                    count_processed+=1
                except Exception as e:
                    print(str(e))
                    pass
        except Exception as e:
            print('\n')
            print('No data!')
            print('This happens in file: ' + str(file))
            print('This is error: ' + str(e))
            print('\n')
            pass


        # Now see how much you can get from the extensions
        try:
            for extension_tweet in twlist['includes']['tweets']:
                count_total+=1
                try:
                    processed_tweet = process_tweet(extension_tweet)
                    try:
                        referenced_tw_id = processed_tweet['referenced_tweets']
                        if referenced_tw_id in file_twit_extension_dict.keys():
                            referenced_tw = file_twit_extension_dict[referenced_tw_id]
                            processed_tweet['referenced_tweets_dictionary']=referenced_tw
                        if referenced_tw_id in file_twit_dict.keys():
                            referenced_tw = file_twit_dict[referenced_tw_id]
                            processed_tweet['referenced_tweets_dictionary']=referenced_tw
                        else:
                            processed_tweet['referenced_tweets_dictionary']='N/A'
                    except:
                        pass
                    dictionary = json.dumps(processed_tweet,ensure_ascii=False)#.encode('utf8')
                    print(dictionary,file=AItrust_twits_dict)
                    count_processed+=1
                except Exception as e:
                    print(str(e))
                    pass
        except Exception as e:
            print('\n')
            print('No tweet extensions.')
            print('This happens in file: ' + str(file))
            print('This is error: ' + str(e))
            print('\n')
            pass

        # Now Authors
        try:
            for author in twlist['includes']['users']:
                # # dict_keys(['description', 'public_metrics', 'created_at', 'id', 'entities', 'name', 'username', 'verified'])
                count_authors+=1
                try:
                    author_dictionary = json.dumps(author,ensure_ascii=False)#.encode('utf8')
                    print(author_dictionary,file=AItrust_author_dict)
                    count_processed_authors+=1
                except Exception as e:
                    print(str(e))
        except Exception as e:
            print('\n')
            print('No Authors?!!')
            print('This happens in file: ' + str(file))
            print('This is error: ' + str(e))
            print('\n')
            pass

AItrust_twits_dict.close()
AItrust_author_dict.close()

print('\n')
print('Tweets counts:')
print(count_processed)
print(count_total)
print('\n')
print('Author counts:')
print(count_processed_authors)
print(count_authors)

  5%|▌         | 257/5016 [09:54<2:25:57,  1.84s/it]



No tweet extensions.
This happens in file: tweets_2023-02-12T00:30:00.json
This is error: 'tweets'




  9%|▉         | 448/5016 [16:38<2:45:22,  2.17s/it]



No tweet extensions.
This happens in file: tweets_2023-02-16T00:00:00.json
This is error: 'tweets'




 15%|█▍        | 736/5016 [27:51<2:32:11,  2.13s/it]

...



No tweet extensions.
This happens in file: tweets_2022-11-26T04:30:00.json
This is error: 'tweets'




 96%|█████████▌| 4801/5016 [2:36:19<07:23,  2.07s/it]



No tweet extensions.
This happens in file: tweets_2022-12-02T00:00:00.json
This is error: 'tweets'




100%|██████████| 5016/5016 [2:44:10<00:00,  1.96s/it]



Tweets counts:
36560405
36560405


Author counts:
31989322
31989322


# Disconnect from Runtime

In [ ]:
from datetime import datetime
import pytz
from IPython.display import Javascript

# Get current time in New York
nyc_time = datetime.now(pytz.timezone('America/New_York'))
formatted_time = nyc_time.strftime('%Y-%m-%d %H:%M:%S %Z')

# Print and log
print(f"✅ Disconnected from runtime at: {formatted_time}")

# Disconnect Colab runtime
display(Javascript('google.colab.kernel.disconnect()'))

✅ Disconnected from runtime at: 2025-08-20 11:07:43 EDT


<IPython.core.display.Javascript object>